# Initialization

In [16]:
# !pip install google-cloud-aiplatform[adk,agent_engines]
# !pip install pytz

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)


In [1]:
import vertexai
from vertexai.preview import reasoning_engines

PROJECT_ID = "hello-world-418507"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://2025-wy-agentspace"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [2]:
import datetime
from zoneinfo import ZoneInfo
from google.adk.agents import Agent

def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city for which to retrieve the weather report.

    Returns:
        dict: status and result or error msg.
    """
    if city.lower() == "new york":
        return {
            "status": "success",
            "report": (
                "The weather in New York is sunny with a temperature of 25 degrees"
                " Celsius (41 degrees Fahrenheit)."
            ),
        }
    else:
        return {
            "status": "error",
            "error_message": f"Weather information for '{city}' is not available.",
        }


def get_current_time(city: str) -> dict:
    """Returns the current time in a specified city.

    Args:
        city (str): The name of the city for which to retrieve the current time.

    Returns:
        dict: status and result or error msg.
    """

    if city.lower() == "new york":
        tz_identifier = "America/New_York"
    else:
        return {
            "status": "error",
            "error_message": (
                f"Sorry, I don't have timezone information for {city}."
            ),
        }

    tz = ZoneInfo(tz_identifier)
    now = datetime.datetime.now(tz)
    report = (
        f'The current time in {city} is {now.strftime("%Y-%m-%d %H:%M:%S %Z%z")}'
    )
    return {"status": "success", "report": report}


root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash-exp",
    description=(
        "Agent to answer questions about the time and weather in a city."
    ),
    instruction=(
        "I can answer your questions about the time and weather in a city."
    ),
    tools=[get_weather, get_current_time],
)

In [3]:

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

## Local test

In [4]:
session = app.create_session(user_id="u_123")
session

Session(id='b8ed8035-2781-4650-918d-778edaf4bef7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1744355476.253941)

In [5]:
app.list_sessions(user_id="u_123")


ListSessionsResponse(sessions=[Session(id='b8ed8035-2781-4650-918d-778edaf4bef7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1744355476.253941)])

In [6]:
session = app.get_session(user_id="u_123", session_id=session.id)
session

Session(id='b8ed8035-2781-4650-918d-778edaf4bef7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1744355476.253941)

In [7]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="whats the weather in london",
):
    print("*"*100)
    print(event)

****************************************************************************************************
{'content': {'parts': [{'function_call': {'id': 'adk-4c465821-8df2-4203-b4a8-86851011bdc8', 'args': {'city': 'london'}, 'name': 'get_weather'}}], 'role': 'model'}, 'invocation_id': 'e-c0642688-c9d8-424e-b4f8-bf1c0b3101b9', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'KfKhOyFh', 'timestamp': 1744355477.186316}
****************************************************************************************************
{'content': {'parts': [{'function_response': {'id': 'adk-4c465821-8df2-4203-b4a8-86851011bdc8', 'name': 'get_weather', 'response': {'status': 'error', 'error_message': "Weather information for 'london' is not available."}}}], 'role': 'user'}, 'invocation_id': 'e-c0642688-c9d8-424e-b4f8-bf1c0b3101b9', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_de

In [8]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="whats the weather in new york",
):
    print("*"*100)
    print(event)

I0000 00:00:1744355481.648136 10516389 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


****************************************************************************************************
{'content': {'parts': [{'function_call': {'id': 'adk-a48bd53d-6c92-43a7-b562-7e38ca649b5e', 'args': {'city': 'new york'}, 'name': 'get_weather'}}], 'role': 'model'}, 'invocation_id': 'e-be75f7d0-4c25-4ab0-a806-f289f4df714d', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'ZA7WswVH', 'timestamp': 1744355480.043027}
****************************************************************************************************
{'content': {'parts': [{'function_response': {'id': 'adk-a48bd53d-6c92-43a7-b562-7e38ca649b5e', 'name': 'get_weather', 'response': {'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25 degrees Celsius (41 degrees Fahrenheit).'}}}], 'role': 'user'}, 'invocation_id': 'e-be75f7d0-4c25-4ab0-a806-f289f4df714d', 'author': 'weather_time_agent'

## Deploy

- reason engine id: 3326620808347910144

In [9]:
from vertexai import agent_engines

remote_app = agent_engines.create(
    agent_engine=root_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]"   
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.88.0'}
The following requirements are missing: {'cloudpickle'}
The following requirements are appended: {'cloudpickle==3.1.1'}
The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'cloudpickle==3.1.1']
Using bucket 2025-wy-agentspace
Wrote to gs://2025-wy-agentspace/agent_engine/agent_engine.pkl
Writing to gs://2025-wy-agentspace/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://2025-wy-agentspace/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/671247654914/locations/us-central1/reasoningEngines/3326620808347910144/operations/442620053712535552
View progress and logs at https://console.cloud.google.com/logs/query?project=hello-world-418507
AgentEngine created. Resource name: projects/671247654914/locations/us-central1/reasoningEn

## Test in new session

In [10]:
live_app = vertexai.agent_engines.get('projects/671247654914/locations/us-central1/reasoningEngines/3326620808347910144')
session = live_app.create_session(user_id="u_123")
session

{'events': [],
 'user_id': 'u_123',
 'state': {},
 'id': '8899056238835269632',
 'app_name': '3326620808347910144',
 'last_update_time': 1744356819.084337}

In [14]:
for event in live_app.stream_query(
    user_id="u_123",
    session_id=session["id"],
    message="whats the weather in london",
):
    print("*"*100)
    print(event)

****************************************************************************************************
{'content': {'parts': [{'function_call': {'id': 'adk-86ef1af9-e475-4ba3-83e5-829c91845935', 'args': {'city': 'london'}, 'name': 'get_weather'}}], 'role': 'model'}, 'invocation_id': 'e-ac77168f-8d9c-41c9-a422-e44404f0b58a', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'bvUPNmIJ', 'timestamp': 1744356864.197811}
****************************************************************************************************
{'content': {'parts': [{'function_response': {'id': 'adk-86ef1af9-e475-4ba3-83e5-829c91845935', 'name': 'get_weather', 'response': {'status': 'error', 'error_message': "Weather information for 'london' is not available."}}}], 'role': 'user'}, 'invocation_id': 'e-ac77168f-8d9c-41c9-a422-e44404f0b58a', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta

## Logging

```bash
resource.type="aiplatform.googleapis.com/ReasoningEngine"
resource.labels.location="us-central1"
resource.labels.reasoning_engine_id="2590282269272834048"
```

In [58]:
from google.cloud import logging
from datetime import datetime, timedelta
import pytz

def get_recent_logs(engine_id, minutes=5):
    """
    Retrieve logs from a specific Reasoning Engine for the last specified minutes.
    
    Args:
        engine_id (str): The ID of the Reasoning Engine
        minutes (int): Number of minutes to look back for logs
    
    Returns:
        List of log entries
    """
    # Initialize the logging client
    client = logging.Client()
    
    # Calculate the time range in Pacific Time
    vegas_tz = pytz.timezone('America/Los_Angeles')
    end_time = datetime.now(vegas_tz)
    start_time = end_time - timedelta(minutes=minutes)
    
    # Format the start timestamp for the query
    start_time_str = start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    
    # Construct the filter string
    filter_str = (
        'resource.type="aiplatform.googleapis.com/ReasoningEngine" '
        'resource.labels.location="us-central1" '
        f'resource.labels.reasoning_engine_id="{engine_id}" '
        f'timestamp >= "{start_time_str}"'
    )
    
    # Create an iterator for the logs and convert to list
    entries = list(client.list_entries(filter_=filter_str))
    
    print(f"Found {len(entries)} log entries in the last {minutes} minutes")
    
    # Print just the payload for each log entry
    for i, entry in enumerate(entries, 1):
        print(f"\n--- Log Entry {i} ---")
        print(f"Payload: {entry.payload}")



In [59]:
# Get logs for the last 15 minutes
get_recent_logs(engine_id="3326620808347910144", minutes=5)

Found 636 log entries in the last 5 minutes

--- Log Entry 1 ---
Payload: INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)

--- Log Entry 2 ---
Payload: INFO:     Started parent process [1]

--- Log Entry 3 ---
Payload: 04/11/2025 07:13:58 AM: Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>

--- Log Entry 4 ---
Payload: 04/11/2025 07:13:58 AM: Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>

--- Log Entry 5 ---
Payload: 04/11/2025 07:13:58 AM: Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>

--- Log Entry 6 ---
Payload: 04/11/2025 07:13:58 AM: Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>

--- Log Entry 7 ---
Payload: 04/11/202

## Agentspace Registration

In [19]:
%%bash
export PROJECT_ID="hello-world-418507" 
export PROJECT_NUMBER="671247654914" 
export REASONING_ENGINE="projects/671247654914/locations/us-central1/reasoningEngines/3326620808347910144" 
export AGENT_DISPLAY_NAME="Wei WeatherAgent"
export AGENT_DESCRIPTION="Agent to answer questions about the time and weather in a city."
export AGENT_ID="wei_weather_agent" 
export AS_APP="2025-traveloka-gds_1741922567491" 

curl -X PATCH -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "x-goog-user-project: ${PROJECT_ID}" \
https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant?updateMask=agent_configs -d '{
    "name": "projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant",
    "displayName": "Default Assistant",
    "agentConfigs": [{
      "displayName": "'"${AGENT_DISPLAY_NAME}"'",
      "vertexAiSdkAgentConnectionInfo": {
        "reasoningEngine": "'"${REASONING_ENGINE}"'"
      },
      "toolDescription": "'"${AGENT_DESCRIPTION}"'",
      "icon": {
        "uri": "https://fonts.gstatic.com/s/i/short-term/release/googlesymbols/corporate_fare/default/24px.svg"
      },
      "id": "'"${AGENT_ID}"'"
    }]
  }'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1369    0   694  100   675    967    941 --:--:-- --:--:-- --:--:--  1906


{
  "name": "projects/671247654914/locations/global/collections/default_collection/engines/2025-traveloka-gds_1741922567491/assistants/default_assistant",
  "displayName": "Default Assistant",
  "agentConfigs": [
    {
      "displayName": "Wei WeatherAgent",
      "vertexAiSdkAgentConnectionInfo": {
        "reasoningEngine": "projects/671247654914/locations/us-central1/reasoningEngines/3326620808347910144"
      },
      "toolDescription": "Agent to answer questions about the time and weather in a city.",
      "icon": {
        "uri": "https://fonts.gstatic.com/s/i/short-term/release/googlesymbols/corporate_fare/default/24px.svg"
      },
      "id": "wei_weather_agent"
    }
  ]
}


## Delete Agent Engine

In [ ]:
remote_app.delete(force=True)